In [ ]:
df = pd.read_csv("../data/features_with_interactions.csv")  # Updated features dataset with interactions


In [2]:
# 📌 Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

# 📌 Step 2: Load Dataset with Feature Interactions
df = pd.read_csv("../data/features_with_interactions.csv")  # Updated features dataset with interactions
y = pd.read_csv("../data/hidden_churn_labels.csv")["Churn"]  # Load hidden churn labels

# 📌 Step 3: Identify Categorical & Numerical Features
categorical_features = ['Gender', 'Income_Group', 'Customer_Type', 'Residency_Status', 'Account_Type', 'Credit_Score']
numerical_features = [col for col in df.columns if col not in categorical_features and col != "Customer_ID"]

# 📌 Step 4: Data Preprocessing Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Apply transformation
X_transformed = preprocessor.fit_transform(df.drop(columns=["Customer_ID"]))
feature_names = numerical_features + list(preprocessor.transformers_[1][1].get_feature_names_out(categorical_features))

# 📌 Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype(np.float32))
X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)  # ✅ Fix Shape Mismatch
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)  # ✅ Fix Shape Mismatch

# 📌 Step 6: Define MLP Model
class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # ✅ Ensures output is between [0,1]
        )
    
    def forward(self, x):
        return self.model(x)

# Initialize model
input_size = X_train.shape[1]
mlp_model = MLPClassifier(input_size)

# 📌 Step 7: Define Loss & Optimizer
criterion = nn.BCELoss()  # ✅ Binary Cross-Entropy Loss for Classification
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# 📌 Step 8: Train the MLP Model
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 20
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_X).view(-1, 1)  # ✅ Fix Shape Mismatch
        batch_y = batch_y.view(-1, 1)  # ✅ Ensure same dtype & shape
        loss = criterion(outputs, batch_y)  # ✅ Correct Loss Computation
        loss.backward()
        optimizer.step()

# 📌 Step 9: Evaluate the Model
with torch.no_grad():
    y_pred_proba = mlp_model(X_test_tensor).view(-1).numpy()  # ✅ Fix Shape Issue
    y_pred = (y_pred_proba > 0.5).astype(int)

# Compute Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"📊 Model Performance (MLP):")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")

print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))

# 📌 Step 10: SHAP Values Calculation
explainer = shap.Explainer(mlp_model, X_train_tensor)
shap_values = explainer(X_test_tensor)

# 📌 Step 11: SHAP Summary Plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values.values, X_test, feature_names=feature_names)


RuntimeError: all elements of input should be between 0 and 1

In [4]:
# 📌 Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

# 📌 Step 2: Load Dataset with Feature Interactions
df = pd.read_csv("../data/features.csv")  # Use the correct dataset
y = pd.read_csv("../data/hidden_churn_labels.csv")["Churn"]

# 📌 Step 3: Identify Categorical & Numerical Features
categorical_features = ['Gender', 'Income_Group', 'Customer_Type', 'Residency_Status', 'Account_Type', 'Credit_Score']
numerical_features = [col for col in df.columns if col not in categorical_features and col != "Customer_ID"]

# 📌 Step 4: Data Preprocessing Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Apply transformation
X_transformed = preprocessor.fit_transform(df.drop(columns=["Customer_ID"]))
feature_names = numerical_features + list(preprocessor.transformers_[1][1].get_feature_names_out(categorical_features))

# 📌 Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype(np.float32))
X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)  
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)  

# 📌 Fix 1: Handle NaN values in tensor data
X_train_tensor = torch.nan_to_num(X_train_tensor, nan=0.0)
y_train_tensor = torch.nan_to_num(y_train_tensor, nan=0.0)

# 📌 Fix 2: Adjust Learning Rate (Avoid Exploding Gradients)
learning_rate = 0.0005  

# 📌 Fix 3: Initialize Model Weights Properly
def initialize_weights(model):
    for layer in model.children():
        if isinstance(layer, nn.Linear):
            nn.init.xavier_uniform_(layer.weight)  # Xavier Initialization
            layer.bias.data.fill_(0.01)

# 📌 Step 6: Define MLP Model
class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # Ensure output values are between [0,1]
        )
    
    def forward(self, x):
        return self.model(x)

# Initialize model
input_size = X_train.shape[1]
mlp_model = MLPClassifier(input_size)
initialize_weights(mlp_model)  # Apply weight initialization

# 📌 Step 7: Define Loss & Optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)

# 📌 Step 8: Train the MLP Model
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_X).view(-1, 1)  # Ensure shape matches
        batch_y = batch_y.view(-1, 1)  # Ensure target shape matches
        loss = criterion(outputs, batch_y)  # Compute loss correctly
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # Print loss for debugging
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

# 📌 Step 9: Evaluate the Model
with torch.no_grad():
    y_pred_proba = mlp_model(X_test_tensor).view(-1).numpy()  # Ensure correct shape
    y_pred = (y_pred_proba > 0.5).astype(int)

# Compute Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"\n📊 Model Performance (MLP):")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")
print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))

# 📌 Step 10: SHAP Values Calculation
explainer = shap.Explainer(mlp_model, X_train_tensor)
shap_values = explainer(X_test_tensor)

# 📌 Step 11: SHAP Summary Plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values.values, X_test, feature_names=feature_names)


Epoch [1/20], Loss: 0.5910
Epoch [2/20], Loss: 0.5870
Epoch [3/20], Loss: 0.5860
Epoch [4/20], Loss: 0.5859
Epoch [5/20], Loss: 0.5853
Epoch [6/20], Loss: 0.5849
Epoch [7/20], Loss: 0.5848
Epoch [8/20], Loss: 0.5845
Epoch [9/20], Loss: 0.5842
Epoch [10/20], Loss: 0.5842
Epoch [11/20], Loss: 0.5841
Epoch [12/20], Loss: 0.5837
Epoch [13/20], Loss: 0.5835
Epoch [14/20], Loss: 0.5835
Epoch [15/20], Loss: 0.5833
Epoch [16/20], Loss: 0.5829
Epoch [17/20], Loss: 0.5828
Epoch [18/20], Loss: 0.5828
Epoch [19/20], Loss: 0.5822
Epoch [20/20], Loss: 0.5822

📊 Model Performance (MLP):
✅ Accuracy: 0.7283
✅ Precision: 0.0000
✅ Recall: 0.0000
✅ F1 Score: 0.0000
✅ ROC-AUC Score: 0.5000

🔍 Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84     14566
           1       0.00      0.00      0.00      5434

    accuracy                           0.73     20000
   macro avg       0.36      0.50      0.42     20000
weighted avg       0.53 

C:\Users\pullu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\pullu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\pullu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

TypeError: 'Tensor' object is not callable

In [5]:
# 📌 Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

# 📌 Step 2: Load Dataset with Feature Interactions
df = pd.read_csv("../data/features_with_interactions.csv")  
y = pd.read_csv("../data/hidden_churn_labels.csv")["Churn"]

# 📌 Step 3: Identify Categorical & Numerical Features
categorical_features = ['Gender', 'Income_Group', 'Customer_Type', 'Residency_Status', 'Account_Type', 'Credit_Score']
numerical_features = [col for col in df.columns if col not in categorical_features and col != "Customer_ID"]

# 📌 Step 4: Data Preprocessing Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Apply transformation
X_transformed = preprocessor.fit_transform(df.drop(columns=["Customer_ID"]))
feature_names = numerical_features + list(preprocessor.transformers_[1][1].get_feature_names_out(categorical_features))

# 📌 Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype(np.float32))
X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)  
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)  

# ✅ Fix NaNs in Training Data
X_train_tensor = torch.nan_to_num(X_train_tensor, nan=0.0)
y_train_tensor = torch.nan_to_num(y_train_tensor, nan=0.0)

# 📌 Step 6: Fix Class Imbalance with Weighted Loss
class_counts = y_train.value_counts().to_dict()
weight_0 = 1 / class_counts[0]
weight_1 = 1 / class_counts[1]
weights = torch.tensor([weight_0, weight_1], dtype=torch.float32)

# 📌 Step 7: Define MLP Model with Fixes
class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),  # ✅ Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 1),
            nn.Sigmoid()  # ✅ Ensure outputs in [0,1]
        )
    
    def forward(self, x):
        return self.model(x)

# Initialize model
input_size = X_train.shape[1]
mlp_model = MLPClassifier(input_size)

# 📌 Step 8: Weight Initialization
def initialize_weights(model):
    for layer in model.children():
        if isinstance(layer, nn.Linear):
            nn.init.xavier_uniform_(layer.weight)
            layer.bias.data.fill_(0.01)  # ✅ Avoid dead neurons

initialize_weights(mlp_model)

# 📌 Step 9: Define Weighted Loss & Optimizer
criterion = nn.BCELoss(weight=weights[y_train_tensor.long()].squeeze())  
optimizer = optim.Adam(mlp_model.parameters(), lr=0.002)

# 📌 Step 10: Train the MLP Model with Learning Rate Decay & Gradient Clipping
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)  # ✅ Learning Rate Decay

num_epochs = 25
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_X).view(-1, 1)
        batch_y = batch_y.view(-1, 1)

        # ✅ Fix Values Outside [0,1]
        outputs = torch.clamp(outputs, min=1e-7, max=1 - 1e-7)  

        loss = criterion(outputs, batch_y)  
        loss.backward()

        # ✅ Gradient Clipping to Avoid Exploding Gradients
        torch.nn.utils.clip_grad_norm_(mlp_model.parameters(), max_norm=1.0)

        optimizer.step()
    
    scheduler.step()  # ✅ Update learning rate
    
    if epoch % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 📌 Step 11: Evaluate the Model
with torch.no_grad():
    y_pred_proba = mlp_model(X_test_tensor).view(-1).numpy()
    y_pred = (y_pred_proba > 0.5).astype(int)

# Compute Performance Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"📊 Model Performance (MLP with Fixes):")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")

print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))

# 📌 Step 12: SHAP Values Calculation (Fixed)
X_test_numpy = X_test_tensor.numpy()

explainer = shap.Explainer(mlp_model, X_train_tensor)
shap_values = explainer(X_test_numpy)  

# 📌 Step 13: SHAP Summary Plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values.values, X_test_numpy, feature_names=feature_names)


RuntimeError: output with shape [64, 1] doesn't match the broadcast shape [64, 80000]